In [ ]:
import pandas as pd
import numpy as np


In [ ]:
url_sn = "https://www.sidc.be/silso/DATA/SN_d_tot_V2.0.txt"

sn_raw = pd.read_csv(
    url_sn,
    sep=r"\s+",        
    header=None,
    comment="#",       
    usecols=[0, 1, 2, 4],
    engine="python"   
)

sn_raw.head(20)


In [ ]:
sn_raw.columns = ["Year", "Month", "Day", "Sunspot"]
sn_raw["date"] = pd.to_datetime(
    sn_raw[["Year", "Month", "Day"]]
)
sn_raw.loc[sn_raw["Sunspot"] < 0, "Sunspot"] = np.nan
sn_raw.head(20)

In [ ]:
sunspot_data = sn_raw[["date", "Sunspot"]].copy()
sunspot_data.columns = ["date", "sunspot_number"]
sunspot_data.head(20)

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
sunspot_data["date"] = pd.to_datetime(
    sunspot_data["date"]
).dt.date
schema = pa.schema([
    ("date", pa.date32()),
    ("sunspot_number", pa.float64())
])

table = pa.Table.from_pandas(
    sunspot_data,
    schema=schema,
    preserve_index=False
)

pq.write_table(
    table,
    "sunspot_daily.parquet"
)
